# Virtual Proctoring using Amazon Rekognition

These notebook provide a walkthrough of some Amazon Rekognition APIs that are relevant to Virtual Proctoring.

In order to start, it's necessary to create a bucket where to host sample images and videos used by each notebook.

In [10]:
# First, let's get the latest installations of our dependencies
!pip install --upgrade pip
!pip install botocore --upgrade
!pip install boto3 --upgrade
!pip install IPython --upgrade

In [2]:
import boto3
import botocore
import os

# Initialize session with profile
session = boto3.Session(profile_name='aws_keclite')
REGION = session.region_name or 'us-east-1'

s3 = session.client('s3', region_name=REGION)
sts = session.client("sts")

# Get account ID
account_id = sts.get_caller_identity()["Account"]

# Define bucket name
bucket_name = f"test-images-{account_id}"

# Create bucket if it doesn't exist
try:
    s3.head_bucket(Bucket=bucket_name)
except botocore.exceptions.ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == '403':
        print("Private Bucket. Forbidden Access! Ensure the bucket is accessible.")
    elif error_code == '404':
        print(f"Bucket {bucket_name} not found. Creating...")
        if REGION == 'us-east-1':
            s3.create_bucket(Bucket=bucket_name)
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': REGION}
            )

# Upload media files
media = ['leaving.mp4', 'objects.mp4', 'cellphone.jpg', 'identity.jpg', 'looking_at_screen.jpg']
for file in media:
    file_name = os.path.join("media", file)
    try:
        with open(file_name, 'rb') as data:
            print(f"Uploading s3://{bucket_name}/{file}")
            s3.upload_fileobj(data, bucket_name, file)
    except FileNotFoundError:
        print(f"File not found: {file_name}")

print("All done.")


Uploading s3://test-images-891377001208/leaving.mp4
Uploading s3://test-images-891377001208/objects.mp4
Uploading s3://test-images-891377001208/cellphone.jpg
Uploading s3://test-images-891377001208/identity.jpg
Uploading s3://test-images-891377001208/looking_at_screen.jpg
All done.


Now you can proceed with the first lab.